<a href="https://colab.research.google.com/github/PJinss/Section1-Project/blob/main/AI_13%EA%B8%B0_%EB%B0%95%EC%A7%84%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/vgames2.csv')
df

In [ ]:
del df['Unnamed: 0'],df['Publisher']  # del df['열 이름'] // 해당열 날리기. // 나의 생각으로는 게임회사는 중요도가 낮다고 판단되어짐

df

In [ ]:
drop_values = ['M','K'] # 지워낼 문자열을 리스트 형태로 저장.
df = df[~df['NA_Sales'].str.contains('|'.join(drop_values))]
df = df[~df['EU_Sales'].str.contains('|'.join(drop_values))]
df = df[~df['JP_Sales'].str.contains('|'.join(drop_values))]
df = df[~df['Other_Sales'].str.contains('|'.join(drop_values))]

# -- [특정 피쳐] <내의 M K 가 포함된 문자열을 가지고 있는 행을 삭제합니다.

In [ ]:
# 결측치를 확인할때 좋은 코드 // 해당 열의 NaN의 값이 들어간 부분을 더해서 숫자로 보여준다
df.isnull().sum()

In [ ]:
df = df.dropna() # axis=0 은 행을 뜻하며 동시에 default 값이고, axis=1 은 열을 뜻함. // 결측치는 싹다 제거해버림.

In [ ]:
df.info()
df.isnull().sum()

In [ ]:
def year(x):
  if (x>=0) & (x< 20):
    return (2000 + x)
  elif (x>30) & (x <100):
    return (1900 + x)
  else:
    return int(x)

df['Year'] = df['Year'].apply(year)


df.info()

In [ ]:
df['NA_Sales'] = df['NA_Sales'].map(pd.eval).astype(float) 
df['EU_Sales'] = df['EU_Sales'].map(pd.eval).astype(float) 
df['JP_Sales'] = df['JP_Sales'].map(pd.eval).astype(float) 
df['Other_Sales'] = df['Other_Sales'].map(pd.eval).astype(float) 

df.describe()

In [ ]:
# 내가 필요한 데이터 만들기. == 그래프 그릴 피쳐만 모여있는 데이터 > 

In [ ]:
# 그룹화 // 군집화 // 클러스터링 초기단계
num_Platform = df['Platform'].unique() # 해당 컬럼의 값들을 나열시킨다. unique
print(len(num_Platform)) # 값들을 나열시키고 그 개수를 세어본다.
num_Platform # 값들을 출력해본다.

In [ ]:
num_Year = df['Year'].unique()
print(len(num_Year))
num_Year

In [ ]:
num_Genre = df['Genre'].unique()
print(len(num_Genre))
num_Genre

In [ ]:
# 한개의 게임에 대한 판매량(모든 나라의)을 합친것 수치를 나타내보자. << Feature Engineering >>

df['Total_Sales'] = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']

df  # 오른편에 추가된것을 확인할 수 있다.

In [ ]:
# 지역마다 선호하는 게임 장르 나누기
# 1step . 장르와 전체 판매량을 묶기.

df_genre = df.groupby(by=['Genre'])['Total_Sales'].sum()
df_genre

In [ ]:
# step_2 . (장르와 북미 , 장르와 유럽 , 장르와 일본 ,  장르와 그외) 를 각각 묶기.
df_genre_na =  df.groupby(by=['Genre'])['NA_Sales'].sum()
df_genre_na

In [ ]:
df_genre_eu =  df.groupby(by=['Genre'])['EU_Sales'].sum()
df_genre_eu

In [ ]:
df_genre_jp =  df.groupby(by=['Genre'])['JP_Sales'].sum()
df_genre_jp

In [ ]:
df_genre_ot =  df.groupby(by=['Genre'])['Other_Sales'].sum()
df_genre_ot

In [ ]:
df_genreF = df_genre.combine_first(df_genre_na).combine_first(df_genre_eu).combine_first(df_genre_jp).combine_first(df_genre_ot)
print(type(df_genre),df_genre)

mydf = pd.concat([df_genreF,df_genre_eu,df_genre_jp,df_genre_na,df_genre_ot],axis=1) # '시리즈에서 데이터 프레임으로 변환'되어진 애들을 다 합쳐버림.

mydf # 그래프를 그릴 데이터 완성

In [ ]:
sns.set(style='darkgrid')    # seaborn의 스타일을 세팅합니다. 다크그리드
plt.figure(figsize= (20,15)) # 
plt.title('Preferred game genres by Region',fontsize=30) # 내가 만든 그래프의 이름을 짓는다.
plt.xlabel('Genre', fontsize=20)                         # x 선에 이름을 넣는다.
plt.ylabel('Total sales',fontsize=20)                    # y 선에 이름을 넣는다.

# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 그래프 틀을 짯으니 이제 변수만 할당해서 넣자.

sns.lineplot(data=mydf,markers=True); # sns.lineplot(data='',markers=T or F) // seaborn 참조.


In [ ]:
# 파란색 선(전세계 총 판매량)을 보면 1위는 Action , Sports , Shooter으로 볼수 있다. 
# ##################               지역마다 선호하는 장르가 나뉘어짐.      ######################################

# North America(북미), Europe(유럽) , Others(그외 국가) 이 세 그룹은 선호하는 장르가 같다.  1위.Action / 2위.Sports / 3위.Shooter  

# 녹색의 그래프 JP_Sales를 보면  //  RPG의 인기는 일본에서만 특별하다고 볼수 있다.
# Japan은 선호하는 장르가 극히 다르다.   1위 RPG // 2위 Action // 3위 Platform(발판을 밟으면서 진행하는 게임을 그룹화한 용어) 

In [ ]:
# 연도별 인기 트렌드 파악하기  : 원본 데이타를 다시 가공해야함.

# 먼저 pandas.groupby 진행

year_sale_dx = df.groupby(by=['Year','Genre'])['Total_Sales'].sum().reset_index() 

year_sale = year_sale_dx.groupby(by=['Year'])['Total_Sales'].transform(max) == year_sale_dx['Total_Sales']

year_sale_max = year_sale_dx[year_sale].reset_index(drop=True)

genre = year_sale_max['Genre']


In [ ]:
year_sale_dx # groupby [연도 , 장르] [총 판매량].총합 >  연도별 장르의 총 판매량

In [ ]:
### year_sale_dx   # 같은연도를 모아서 나열 # [연도 , 장르] [총 판매량].총합 >  연도별 장르의 총 판매량

# year_sale          # 특정 장르안에서 최대 판매량 == 같으면 True , 다르면 False

year_sale_max  # 연도별 최다 판매량찍은 행만 걸러냄 (해당년도의 1위 게임장르)

In [ ]:
genre = year_sale_max['Genre']

plt.figure(figsize=(30,20))
g = sns.barplot(x='Year',y='Total_Sales',data=year_sale_max) 

# bar에 설명을 넣기.
index = 0 # for문을 위한 숫자를 넣은 변수

for v in year_sale_max['Total_Sales']:
   g.text(index, v + 1, str(genre[index] + '-' + str(round(v,2))), color='#000',size=20,rotation= 90,ha='center' )
   index += 1

plt.title('Game trends by year',fontsize=30)
plt.xlabel('Year',fontsize=20)
plt.ylabel('Total sales',fontsize=20)

plt.xticks(rotation=45) # 45도로 x축 제목을 틀겠다.
plt.show


In [ ]:
## ========================= 시각화를 통한 분석 ===============================

######## 연도별 게임 트렌드 시각화 ############

# 2001년 부터 액션게임의 수요가 급격히 늘어났다.
# 폭발적인 게임의 수요가 증가하던 2006년과 2009년에 스포츠게임의 수요가 엄청났다.
# 2007년 부터는 Action게임의 수요는 

In [ ]:
# 게임중 가장 많이 팔린게임을 골라내고 , 왜 많이 팔렸을지 분석및 시각화 프로세스 

# 필요한 피쳐가 무엇일까? -> Name , Year , Genre , Total_Sales

sortdf = df.sort_values(by=['Total_Sales'], ascending=False)  #값을 기준으로 정렬한다
top_game_sale = sortdf.head(20)
top_game_sale = top_game_sale[['Name','Year','Genre','Total_Sales']]
top_game_sale = top_game_sale.sort_values(by=['Total_Sales'], ascending=False)
top_game_sale = top_game_sale.reset_index(drop=True)


In [ ]:
top_game_sale

In [ ]:
# 이게 도통 무슨 코드인지 모르겠다.
name = top_game_sale['Name']
year = top_game_sale['Year']
y = np.arange(0, 20)

plt.figure(figsize=(30, 18))
g = sns.barplot(x='Name', y='Total_Sales', data=top_game_sale) # 변수에 바플랏을 넣고 해당 바플랏이 나타낼 x축y축 값을 정해준다.data= 사용할 데이터셋

index = 0
for value in top_game_sale['Total_Sales']:
    g.text(index, value - 18, name[index], color='#000', size=14, rotation= 90, ha="center")
    index += 1

plt.xticks(y, top_game_sale['Year'], fontsize=14, rotation=90)
plt.title("The best-selling game on the continent", fontsize=30)
plt.xlabel("Release Year", fontsize=20)
plt.ylabel("Total sales volume", fontsize=20)
plt.show()


In [ ]:
## ================================== 역대급으로 가장 많은 판매량을 기록했던 게임 알아보기 =======================================. ##


# 위의 바플랏을 그림으로써(시각화) 가장 많이 팔린 게임을 알수 있게 되었다.
# 1위 wii sports , 2위는 super mario , 3위는  Mario Kart이다.


